In [1]:
import sqlite3

In [120]:
############################################
#helper methods
def init_db_connection():
    # init db connection
    conn = sqlite3.connect("./shares.db")
    sql = conn.cursor()   
    return conn, sql

def close_db_connection(conn, sql):
    # close connection
    sql.close()
    conn.close()

def get_stocks_from_portfolio(portfolio):
    conn, sql = init_db_connection()
    sql.execute("select shares.name, shares.identifier from portfolio_shares inner join portfolio on portfolio_shares.portfolio=portfolio.id inner join shares on portfolio_shares.id=shares.id;")
    results=sql.fetchall()
    close_db_connection(conn, sql)
    return results    

    
    
def get_ammount_and_price(portfolio_id,stock_id):
    conn,sql = init_db_connection()
    sql.execute("Select anteile, einstandskurs from portfolio_shares where portfolio="+str(portfolio_id)+" and share ="+str(stock_id))
    ammount,price=sql.fetchone()
    close_db_connection(conn,sql)
    return ammount,price


def get_ids(portfolio, stock):
    conn,sql = init_db_connection()
    sql.execute("Select id from portfolio where name='"+portfolio+"'")    
    portfolio_id=sql.fetchone()[0]
    sql.execute("Select id from shares where name='"+stock+"'")    
    stock_id=sql.fetchone()[0]
    sql.execute("Select portfolio_shares.id from portfolio_shares where portfolio="+str(portfolio_id)+" and share ="+str(stock_id))    
    stock_portfolio_id=sql.fetchone()[0]
    close_db_connection(conn,sql)
    return portfolio_id, stock_id, stock_portfolio_id
###############################################

# relevant methods to call

def buy_stock(portfolio, stock, price, ammount): # portfolio and stock = names, price = closing value, ammount = ammount
    conn,sql = init_db_connection()
    portfolio_id, stock_id, portfolio_stock_id=get_ids(portfolio,stock)
    old_stocks, old_price=get_ammount_and_price(portfolio_id,stock_id)
    new_stocks=old_stocks+ammount
    new_price=(old_price*old_stocks+price*ammount)/new_stocks # neuer einstandskurs anteilig errechnet
    sql.execute("update portfolio_shares set anteile="+str(new_stocks)+", einstandskurs="+str(new_price)+" where id ="+str(portfolio_stock_id))   
    conn.commit()
    kapital=get_kapital(portfolio)
    sql.execute("update portfolio set aktuelles_kapital="+str(kapital-price*ammount))
    conn.commit()
    print("Es wurden "+str(ammount)+" " + stock + " Aktien für "+str(price)+" € gekauft")
    close_db_connection(conn, sql)
    # ernidrige kapital!
    
def sell_stock(portfolio, stock, price, ammount): # portfolio and stock = names, price = closing value, ammount = ammount
    conn,sql = init_db_connection()
    portfolio_id, stock_id, portfolio_stock_id=get_ids(portfolio,stock)
    old_stocks, old_price=get_ammount_and_price(portfolio_id,stock_id)
    new_stocks=old_stocks-ammount
    sql.execute("update portfolio_shares set anteile="+str(new_stocks)+" where id ="+str(portfolio_stock_id))   
    conn.commit()
    gewinn = (price-old_price)*ammount
    print("Es wurden "+str(ammount)+" " + stock + " Aktien verkauft")
    print("Gewinn: "+str(gewinn)+" €")
    kapital=get_kapital(portfolio)
    sql.execute("update portfolio set aktuelles_kapital="+str(kapital+price*ammount))
    conn.commit()
    if new_stocks==0:
        sql.execute("update portfolio_shares set einstandskurs=0 where id ="+str(portfolio_stock_id))   
        conn.commit()
    # kapital erhöhen 
    close_db_connection(conn, sql)

def get_kapital(portfolio): # portfolio = name
    conn,sql = init_db_connection()
    portfolio_id, _ , _ =get_ids(portfolio,"Allianz") # Allianz muss immer da sein in shares, sonst error
    sql.execute("Select aktuelles_kapital from portfolio where id="+str(portfolio_id))
    kapital=sql.fetchone()[0]
    close_db_connection(conn,sql)
    return kapital     

def get_anzahl(portfolio,stock): # portfolio and stock = names,
    conn,sql = init_db_connection()
    _, _ , stock_portfolio_id =get_ids(portfolio,stock)
    sql.execute("select anteile from portfolio_shares where id ="+str(stock_portfolio_id))
    kapital=sql.fetchone()[0]
    close_db_connection(conn,sql)
    return kapital    

def get_einstandskurs(portfolio,stock): # portfolio and stock = names,
    conn,sql = init_db_connection()
    _, _ , stock_portfolio_id =get_ids(portfolio,stock)
    sql.execute("select einstandskurs from portfolio_shares where id ="+str(stock_portfolio_id))
    kapital=sql.fetchone()[0]
    close_db_connection(conn,sql)
    return kapital  